色検出

In [1]:
import cv2
import numpy as np
cap = cv2.VideoCapture("movie/Mobility.mp4")
while True:
    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("img", 640,400)
    ret, frame = cap.read()
    if ret == False:
        break
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower = np.array([20,50,50])
    upper = np.array([25, 255, 255])
    frame_mask = cv2.inRange(hsv, lower, upper)
    dst = cv2.bitwise_and(frame, frame, mask = frame_mask)
    cv2.imshow("img", dst)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

オプティカルフロー

In [2]:
import cv2
cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.resizeWindow("img", 1200, 800)
COUNT = 500
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 20, 0.03)
lk_params = dict(winSize=(10,10), maxLevel=4, criteria=criteria)
cap = cv2.VideoCapture("movie/Cosmos.mp4")
ret, frame = cap.read()
frame_pre = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame_now = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    feature_pre = cv2.goodFeaturesToTrack(frame_pre, COUNT, 0.001, 5)
    if feature_pre is None:
        continue
    feature_now, status, err = cv2.calcOpticalFlowPyrLK(frame_pre, frame_now, feature_pre, None, **lk_params)
    for i in range(COUNT):
        pre_x = feature_pre[i][0][0]
        pre_y = feature_pre[i][0][1]
        now_x = feature_pre[i][0][0]
        now_y = feature_pre[i][0][1]
        cv2.line(frame, (pre_x,pre_y), (now_x,now_y), (255,0,0),3)
    cv2.imshow("img",frame)
    frame_pre = frame_now.copy()
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

MeanShift/CamShift

In [3]:
cap = cv2.VideoCapture("movie/Cruse.mp4")
ret, frame = cap.read()
h, w, ch = frame.shape

In [4]:
rct = (600,500,200,200)
cv2.namedWindow("win", cv2.WINDOW_NORMAL)
cv2.resizeWindow("win", 1200, 800)
cri = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1)
while(True):
    threshold = 100
    ret, frame = cap.read()
    if ret == False:
        break
    img_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img_g, threshold, 255, cv2.THRESH_BINARY)
    ret, rct = cv2.meanShift(img_bin, rct, cri)
    x, y, w, h = rct
    frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)
    cv2.imshow("win", frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

In [5]:
rct = (600,500,200,200)
cv2.namedWindow("win", cv2.WINDOW_NORMAL)
cv2.resizeWindow("win", 1200, 800)
cri = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1)
while(True):
    threshold = 100
    ret, frame = cap.read()
    if ret == False:
        break
    img_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img_g, threshold, 255, cv2.THRESH_BINARY)
    ret, rct = cv2.CamShift(img_bin, rct, cri)
    x, y, w, h = rct
    frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)
    cv2.imshow("win", frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

背景差分

In [1]:
cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.resizeWindow("img", 1200, 800)
cap = cv2.VideoCapture("movie/People.mp4")
ret, frame = cap.read()
h, w, ch = frame.shape
frame_back = np.zeros((h,w,ch), dtype=np.float32)
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame_diff = cv2.absdiff(frame.astype(np.float32), frame_back)
    cv2.accumulateWeighted(frame, frame_back, 0.03)
    cv2.imshow("img", frame_diff.astype(np.uint8))
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

パーティクルフィルタ